# Python Notebook for Calculation of Expected and SIR Values

## Preliminaries

Import Dependencies

In [1]:
import pandas as pd

Set Province

In [2]:
province = "CATANDUANES"

Load CSV files

In [3]:
# load cases dataset
df_cases = pd.read_csv(f'../01_data/01_processed/00_case_data/{province}_case_data.csv')
df_cases['Date'] = pd.to_datetime(df_cases['Date'])

# load population dataset
df_pop = pd.read_csv(f'../01_data/01_processed/01_population_data/{province}_simulated_population.csv')

In [4]:
df_cases.head()

,Date,Municipality,NewCases,Deaths,Recoveries,d_cases,n
0,2020-06-06,SAN MIGUEL,1,0,0,1,1
1,2020-06-07,SAN MIGUEL,0,0,0,0,1
2,2020-06-08,SAN MIGUEL,0,0,0,0,1
3,2020-06-09,SAN MIGUEL,0,0,0,0,1
4,2020-06-10,SAN MIGUEL,0,0,0,0,1


In [5]:
df_pop

,Municipality,2020,2021,2022
0,BAGAMANOC,11086,11239,11393
1,BARAS,13484,13274,13064
2,BATO,21748,21593,21438
3,CARAMORAN,32114,32114,32114
4,GIGMOTO,8712,8712,8712
5,PANDAN,21473,21157,20841
6,PANGANIBAN,9713,9713,9713
7,SAN ANDRES,38480,38480,38480
8,SAN MIGUEL,15680,15458,15235
9,VIGA,22869,22458,22047


Set date range to be used

In [6]:
start_date = pd.to_datetime('2021-01-01')
end_date = pd.to_datetime('2022-12-31')

(start_date, end_date)

(Timestamp('2021-01-01 00:00:00'), Timestamp('2022-12-31 00:00:00'))

Filter dataframe to only use select dates

In [7]:
df_cases_filtered = df_cases[(df_cases["Date"] >= start_date) & (df_cases["Date"] <= end_date)]
df_cases_filtered

,Date,Municipality,NewCases,Deaths,Recoveries,d_cases,n
209,2021-01-01,SAN MIGUEL,0,0,0,0,0
210,2021-01-02,SAN MIGUEL,0,0,0,0,0
211,2021-01-03,SAN MIGUEL,0,0,0,0,0
212,2021-01-04,SAN MIGUEL,0,0,0,0,0
213,2021-01-05,SAN MIGUEL,0,0,0,0,0
...,...,...,...,...,...,...,...
14054,2022-12-27,VIGA,0,0,0,0,0
14055,2022-12-28,VIGA,0,0,0,0,0
14056,2022-12-29,VIGA,0,0,0,0,0
14057,2022-12-30,VIGA,0,0,0,0,0


## Formula

The SIR value was calculated by Satorra & Tebé (2024) using this formla:
$$
    SIR_{it} = Y_{it}/E_{i}
$$
Where $Y_{it}$ is the observed number of cases in the $i$-th ABS (spatial division) and $E_i$ is the expected number of cases if it behaved like the whole population for the entire period.

The expected number of cases $E_i$ were calculated using indirect standardization with this formula:
$$
    E_i=\sum_{j=1}^J r_jN_j
$$
Where $N_j$ is the population in the $j$-th sex-age stratum of the area and $r_j$ is the average rate for the whole period in the same stratum and:
$$
    r_j=\frac{\sum_{i=1}^n\sum_{t=1}^T Y_{ijt}}{\sum_{i=1}^n\sum_{t=1}^T N_{ijt}}
$$

## Simplification of Formula

Since there is no available data for population by age and sex for each municipality, simplify the formula for $E_i$ to only take into account the expected number of cases for the entire population of the municipality:
$$
    E_i=rN_i
$$
Where $N_i$ is the total population of the $i$-th area and $r$ is the average rate for the whole period in the entire area.
[TO ADD: FORMULA]

`i` refers to the town. `t` is yung time period. `j` yung stratum.

What if
$$
r_i =\frac{\sum_{t=1}^T Y_{it}}{\sum_{t=1}^T N_{it}}
$$
then
$$
E_i = r_i\times N_i
$$

## Calculation of Average Rate $r$

We transform the original equation by removing age-sex stratum and transforming `r` to be specific on one area.

Original equation:
$$
    r_j=\frac{\sum_{i=1}^n\sum_{t=1}^T Y_{ijt}}{\sum_{i=1}^n\sum_{t=1}^T N_{ijt}}
$$
Stratum removed:
$$
    r =\frac{\sum_{i=1}^n\sum_{t=1}^T Y_{it}}{\sum_{i=1}^n\sum_{t=1}^T N_{it}}
$$
Municipality specific:
$$
    r_i =\frac{\sum_{t=1}^T Y_{it}}{\sum_{t=1}^T N_{it}}
$$

From here, we'll have:
$$
E_{it} = r_i \times N_{it}
$$
where we compute for the expected value for each municipality in a given time frame.

Now, we'll have:

$$
SIR_{i} = \frac{\sum_{t=1}^{T} Y_{it}}{\sum_{t=1}^{T} E_{it}}
$$

In [29]:
municipalities = df_cases_filtered['Municipality'].unique().tolist()

r = {}

year_range = range(int(start_date.year), int(end_date.year)+1)

for mun in municipalities:
    num, denum = 0, 0
    for year in year_range:
        num += df_cases_filtered[(df_cases_filtered['Date'].dt.year == year) & \
                                   (df_cases_filtered['Municipality'] == mun)]['n'].sum()
        denum += df_pop[df_pop['Municipality'] == mun][str(year)].values[0]
    print(f'{mun}: {num}, {denum}')
    r[mun] = num/denum

SAN MIGUEL: 1854, 30693
BAGAMANOC: 1835, 22632
VIRAC: 15192, 150199
CARAMORAN: 2989, 64228
GIGMOTO: 1335, 17424
BARAS: 2296, 26338
PANGANIBAN: 1629, 19426
PANDAN: 2476, 41998
BATO: 3903, 43031
SAN ANDRES: 6584, 76960
VIGA: 2564, 44505


In [27]:
expected_values = {}

for mun in municipalities:
    expected_values[mun] = {}

for year in year_range:
    for mun in municipalities:
        expected_values[mun][year] = r[mun]*df_pop[df_pop['Municipality'] == mun][str(year)].values[0]

In [28]:
import itertools

def return_expected_value(mun_year):
    mun, year = mun_year['Municipality'], mun_year['Year']
    return expected_values[mun][year]

mun_year_combination = list(itertools.product(municipalities, year_range))

df_exp = pd.DataFrame(mun_year_combination, columns=['Municipality', 'Year'])

df_exp['EV'] = df_exp.apply(return_expected_value, axis=1)

df_exp

,Municipality,Year,EV
0,SAN MIGUEL,2021,933.735119
1,SAN MIGUEL,2022,920.264881
2,BAGAMANOC,2021,911.256849
3,BAGAMANOC,2022,923.743151
4,VIRAC,2021,7643.892543
5,VIRAC,2022,7548.107457
6,CARAMORAN,2021,1494.500000
7,CARAMORAN,2022,1494.500000
8,GIGMOTO,2021,667.500000
9,GIGMOTO,2022,667.500000


In [17]:
sir = {}

for mun in municipalities:
    num = 0
    for year in year_range:
        num += df_cases_filtered[(df_cases_filtered['Date'].dt.year == year) & \
                                   (df_cases_filtered['Municipality'] == mun)]['n'].sum()
    sir[mun] = num/expected_values[mun][year]

Have SIR here.

---------------------

In [92]:
r = 0

year_range = range(int(start_date.year), int(end_date.year)+1)

# calculate rate per year
for year in year_range:
    # get sum per year
    r += (df_cases_filtered[df_cases_filtered['Date'].dt.year == year]['n']/ df_pop[f'{year}'].sum()).sum()

    #print((df_cases_filtered[df_cases_filtered['Date'].dt.year == year]['n']/ df_pop[f'{year}'].sum()).sum())
    

#r = r / len(year_range)
r

0.1585127054024255

$$
r = \sum_{t=1}^T \frac{Y_t}{N_t}
$$
$$
Y_t
$$
denotes the number of observed cases in year `t` while
$$
N_t
$$
is the population of Catanduanes at year `t`.

Create new DataFrame to calcute for expected number of cases

In [93]:
df_exp = pd.DataFrame(columns=['Municipality', 'exp'])


for mun in df_pop['Municipality'].unique():
    # calculate expected number of cases, E_i = r*N_i
    N_i = 0

    for year in year_range:
        N_i += df_pop[df_pop['Municipality'] == mun][f'{year}']
    
    # expected number of cases is equal to the average number of cases per day per municipality
    E_i = (r * (N_i/ len(year_range))) / len(pd.date_range(start=start_date, end=end_date))

    # add expected value to row
    df_exp.loc[-1] = [mun, E_i.iloc[0]]
    df_exp.index = df_exp.index + 1 
    df_exp = df_exp.sort_index()


df_exp
        


,Municipality,exp
0,VIRAC,16.307157
1,VIGA,4.831923
2,SAN MIGUEL,3.332350
3,SAN ANDRES,8.355574
4,PANGANIBAN,2.109088
5,PANDAN,4.559737
6,GIGMOTO,1.891730
7,CARAMORAN,6.973256
8,BATO,4.671891
9,BARAS,2.859526


Combine results with DataFrame

In [94]:
df_cases_filtered

,Date,Municipality,NewCases,Deaths,Recoveries,d_cases,n
209,2021-01-01,SAN MIGUEL,0,0,0,0,0
210,2021-01-02,SAN MIGUEL,0,0,0,0,0
211,2021-01-03,SAN MIGUEL,0,0,0,0,0
212,2021-01-04,SAN MIGUEL,0,0,0,0,0
213,2021-01-05,SAN MIGUEL,0,0,0,0,0
...,...,...,...,...,...,...,...
14054,2022-12-27,VIGA,0,0,0,0,0
14055,2022-12-28,VIGA,0,0,0,0,0
14056,2022-12-29,VIGA,0,0,0,0,0
14057,2022-12-30,VIGA,0,0,0,0,0


In [95]:
df_cases_filtered_exp = pd.merge(df_cases_filtered, df_exp, on='Municipality')
df_cases_filtered_exp

,Date,Municipality,NewCases,Deaths,Recoveries,d_cases,n,exp
0,2021-01-01,SAN MIGUEL,0,0,0,0,0,3.332350
1,2021-01-02,SAN MIGUEL,0,0,0,0,0,3.332350
2,2021-01-03,SAN MIGUEL,0,0,0,0,0,3.332350
3,2021-01-04,SAN MIGUEL,0,0,0,0,0,3.332350
4,2021-01-05,SAN MIGUEL,0,0,0,0,0,3.332350
...,...,...,...,...,...,...,...,...
8025,2022-12-27,VIGA,0,0,0,0,0,4.831923
8026,2022-12-28,VIGA,0,0,0,0,0,4.831923
8027,2022-12-29,VIGA,0,0,0,0,0,4.831923
8028,2022-12-30,VIGA,0,0,0,0,0,4.831923


Export to New CSV File

In [ ]:
df_cases_filtered_exp.to_csv(f"../01_data/01_processed/00_case_data/{province}_case_data_exp.csv",  index=False)